In [1]:
example_input = """
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
"""

def parse_instructions(lines):
    instructions = []
    for ln in lines:
        instr,arg = ln.split()
        instructions.append((instr, int(arg)))
    return instructions

def execute_break_loop(instructions, debug=False, silent=False, tryFix=False, pos=0, acc=0, stack=None):
    executed = stack if stack else []
    while pos not in executed and pos < len(instructions):
        instr, arg = instructions[pos]
        if debug:
            print(f"DEBUG: Executing #{pos:d}: {instr} {arg:d}")
        if instr == "nop":
            nPos = pos+1
        elif instr == "acc":
            acc += arg
            nPos = pos+1
        elif instr == "jmp":
            nPos = pos+arg
        executed.append(pos)
        pos = nPos
    if pos in executed and tryFix:
        if not silent:
            print("Detected loop, stepping back and trying to fix")
        rAcc = acc
        for i,rPos in enumerate(reversed(executed)):
            rInstr, rArg = instructions[rPos]
            if rInstr == "nop":
                tryInstr = list(instructions)
                tryInstr[rPos] = ("jmp", rArg)
                if debug:
                    print(f"Trying to fix by changing line {rPos:d} from '{rInstr} {rArg:d}' to 'jmp {rArg:d}'")
                tPos, tAcc = execute_break_loop(tryInstr, silent=(not debug), tryFix=False,
                                                pos=rPos, acc=rAcc, stack=executed[:-1-i])
                if tPos == len(instructions):
                    pos, acc = tPos, tAcc
                    break
            elif rInstr == "jmp":
                tryInstr = list(instructions)
                tryInstr[rPos] = ("nop", rArg)
                if debug:
                    print(f"Trying to fix by changing line {rPos:d} from '{rInstr} {rArg:d}' to 'nop {rArg:d}'")
                tPos, tAcc = execute_break_loop(tryInstr, silent=(not debug), tryFix=False,
                                                pos=rPos, acc=rAcc, stack=executed[:-1-i])                    
                if tPos == len(instructions):
                    pos, acc = tPos, tAcc
                    break
            elif rInstr == "acc":
                rAcc -= rArg
    if not silent:
        if pos in executed:
            print(f"Executing instruction {pos:d} for a second time. Accumulator value is {acc:d}")
        elif pos == len(instructions):
            print(f"Terminated successfully! Accumulator is {acc:d}")
    return pos, acc

example_instructions = parse_instructions((ln.strip() for ln in example_input.split("\n") if ln.strip()))
execute_break_loop(example_instructions, debug=True)

DEBUG: Executing #0: nop 0
DEBUG: Executing #1: acc 1
DEBUG: Executing #2: jmp 4
DEBUG: Executing #6: acc 1
DEBUG: Executing #7: jmp -4
DEBUG: Executing #3: acc 3
DEBUG: Executing #4: jmp -3
Executing instruction 1 for a second time. Accumulator value is 5


(1, 5)

In [2]:
with open("inputs/day08.txt") as inF:
    puzzle_instructions = parse_instructions((ln.strip() for ln in inF if ln.strip()))

execute_break_loop(puzzle_instructions)

Executing instruction 228 for a second time. Accumulator value is 2034


(228, 2034)

In [3]:
example_input_fixed = """
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
nop -4
acc +6
"""

example_instructions_fixed = parse_instructions((ln.strip() for ln in example_input_fixed.split("\n") if ln.strip()))
execute_break_loop(example_instructions_fixed, debug=True)

DEBUG: Executing #0: nop 0
DEBUG: Executing #1: acc 1
DEBUG: Executing #2: jmp 4
DEBUG: Executing #6: acc 1
DEBUG: Executing #7: nop -4
DEBUG: Executing #8: acc 6
Terminated successfully! Accumulator is 8


(9, 8)

In [4]:
execute_break_loop(example_instructions, debug=True, tryFix=True)

DEBUG: Executing #0: nop 0
DEBUG: Executing #1: acc 1
DEBUG: Executing #2: jmp 4
DEBUG: Executing #6: acc 1
DEBUG: Executing #7: jmp -4
DEBUG: Executing #3: acc 3
DEBUG: Executing #4: jmp -3
Detected loop, stepping back and trying to fix
Trying to fix by changing line 4 from 'jmp -3' to 'nop -3'
Executing instruction 6 for a second time. Accumulator value is -94
Trying to fix by changing line 7 from 'jmp -4' to 'nop -4'
Terminated successfully! Accumulator is 8
Terminated successfully! Accumulator is 8


(9, 8)

In [5]:
execute_break_loop(puzzle_instructions, debug=False, tryFix=True)

Detected loop, stepping back and trying to fix
Terminated successfully! Accumulator is 672


(631, 672)